# Import libraries

In [7]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import re
import pdfplumber
from gensim.models.fasttext import FastText as FT_gensim
# nltk.download('punkt') 


# Define tokenization and preprocessing functions

In [8]:
def tokeniser(texte):
    words = word_tokenize(texte)
    stemmer_en = SnowballStemmer("english")
    stemmer_fr = SnowballStemmer("french")
    stopwords_fr_en = stopwords.words("english") + stopwords.words("french")
    words = [word for word in words if word.lower() not in stopwords_fr_en and word not in string.punctuation]
    stemmed_words = [stemmer_fr.stem(word) for word in words] + [stemmer_en.stem(word) for word in words]
    return set(stemmed_words)

def appliquer_tokenisation_et_filtrage(texte, competences_tokenized_flat):
    tokens = tokeniser(texte)
    return [token for token in tokens if token in competences_tokenized_flat]

def preparer_liste_competences(liste_competences):
    return {item for sublist in liste_competences for item in tokeniser(" ".join(sublist))}

def filtrer_competences_df(df, liste_competences):
    competences_tokenized_flat = preparer_liste_competences(liste_competences)
    df['competence_processed'] = df['description'].apply(lambda x: appliquer_tokenisation_et_filtrage(x, competences_tokenized_flat))
    return df

def filtrer_competences_texte(texte, liste_competences):
    competences_tokenized_flat = preparer_liste_competences(liste_competences)
    return appliquer_tokenisation_et_filtrage(texte, competences_tokenized_flat)

def lirePDF(chemin_acces):
    # Extraire le texte du PDF
    with pdfplumber.open(chemin_acces) as pdf:
        texte = ""
        for page in pdf.pages:
            texte += page.extract_text()

    texte= re.sub(r'http\S+', ' ', texte) # enlever liens(texte)

    texte= re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', texte) # enlever adresses email

    texte= re.sub(r'[0-9()+-]', ' ', texte) #enlever numeros et symboles

    texte= re.sub(r'[^\w\s]', ' ', texte) # enlever symboles
    
    return texte

# Process DataFrame and specific text

In [9]:
liste_competences_rome = [['langage développement web HTML CSS JavaScript Python PHP Ruby Java C#',
  'framework développement web Django Flask Express Angular React Vue.js Laravel Ruby on Rails ASP.NET',
  'base de données SQL MySQL PostgreSQL MongoDB Firebase',
  "conception d'interfaces utilisateur",
  "intégration d'API RESTful",
  'optimisation des performances web',
  'sécurité des applications web',
  "tests unitaires et tests d'intégration",
  'gestion de versions Git',
  'résolution de problèmes techniques',
  'développement web',
  'design responsive',
  'SEO'],
 ['langage programmation Java Python C++ C# JavaScript Ruby Swift Kotlin',
  'frameworks logiciel Spring Hibernate .NET Django Ruby on Rails React Angular Vue.js',
  "développement d'applications desktop et mobile",
  'architecture logicielle',
  "tests unitaires et tests d'intégration",
  'conception orientée objet',
  'gestion de bases de données relationnelles et NoSQL',
  'optimisation des performances logicielles',
  'résolution de bugs et débogage',
  'gestion de projets logiciels',
  'architecture microservices',
  'Intégration continue',
  'Livraison continue'],
 ['langage programmation Python R SQL Scala Julia MATLAB',
  'outils de manipulation de données Pandas NumPy SciPy',
  'bibliothèques de Machine Learning et Deep Learning TensorFlow PyTorch Scikit-learn Keras',
  'visualisation de données Matplotlib Seaborn Plotly Tableau',
  'analyse statistique',
  'modélisation prédictive',
  'traitement du langage naturel',
  'apprentissage automatique non supervisé',
  'gestion de bases de données Big Data',
  'communication des résultats',
  'Big Data',
  'data engineering',
  'cloud data services'],
 ['sécurité informatique',
  'tests de pénétration',
  'gestion des identités et des accès',
  'sécurité des réseaux',
  'cryptographie',
  'analyse des vulnérabilités',
  "détection d'intrusions",
  'gestion des incidents de sécurité',
  'conformité réglementaire',
  'formation à la sécurité',
  'forensics informatique',
  'sécurité des endpoints',
  'sécurité mobile'],
 ["administration de systèmes d'exploitation Linux Windows Unix",
  'configuration de réseaux locaux et étendus',
  'gestion des services réseau (DNS, DHCP, LDAP)',
  'sécurité des réseaux et des pare-feu',
  'virtualisation et conteneurisation',
  'sauvegarde et restauration de données',
  'supervision et surveillance des réseaux',
  'dépannage réseau',
  'documentation des infrastructures',
  'collaboration avec les équipes techniques',
  'SDN',
  'Zero Trust network',
  'automatisation réseau'],
 ['langage développement mobile Java Kotlin Swift',
  'framework développement mobile Android Studio Xcode',
  "conception d'interfaces utilisateur mobiles",
  "intégration d'API et services web",
  "tests unitaires et tests d'intégration",
  'débogage et optimisation des performances',
  'gestion des versions et déploiements',
  'veille technologique en développement mobile',
  'collaboration avec des équipes de développement',
  'Flutter',
  'React Native',
  'sécurité des applications mobiles'],
 ['conception architecture systèmes informatiques',
  'analyse des besoins et des exigences',
  'sélection de technologies et plateformes',
  'mise en œuvre de solutions évolutives',
  'sécurité et intégrité des données',
  'gestion des performances et de la capacité',
  'interopérabilité et intégration de systèmes',
  'documentation des architectures',
  'gestion des risques et conformité aux normes',
  'collaboration avec des équipes multidisciplinaires',
  'cloud hybride',
  'multi-cloud',
  'conteneurs'],
 ['conception interfaces utilisateur intuitives',
  'analyse des besoins des utilisateurs',
  'création de wireframes et maquettes',
  "tests d'utilisabilité et rétroaction",
  'compétences en design graphique',
  'adaptabilité aux tendances en UX/UI',
  'communication des concepts de design',
  'respect des délais et gestion de projet',
  'collaboration avec des équipes de développement et marketing',
  'design thinking',
  'Figma',
  'accessibilité web'],
 ['analyse besoins métier et processus',
  'documentation des exigences fonctionnelles et non fonctionnelles',
  'modélisation des processus métier et flux de données',
  'évaluation des systèmes existants',
  'coordination entre équipes métier et techniques',
  "tests d'acceptation utilisateur",
  'support technique et résolution des problèmes',
  'formation des utilisateurs finaux',
  'reporting et suivi des indicateurs de performance',
  'adaptabilité aux changements et évolutions des besoins',
  'Agile',
  'Scrum',
  'DevOps'],
 ['conception, déploiement et gestion architectures cloud',
  'automatisation des opérations (Infrastructure as Code)',
  'sécurité et conformité des données',
  'optimisation des coûts et performances',
  'haute disponibilité et reprise après sinistre',
  "migration d'applications vers le cloud",
  'gestion des identités et des accès (IAM)',
  'intégration de services cloud (IaaS, PaaS, SaaS)',
  'monitoring et surveillance des services cloud',
  "collaboration avec des équipes de développement et d'exploitation",
  'serverless',
  'architecture orientée services',
  'cloud hybride et multi-cloud'],
 ['soins infirmiers',
  'diagnostic et évaluation des patients',
  'administration de traitements médicaux',
  'suivi et surveillance des patients',
  'communication avec les patients et leurs familles',
  "coordination des soins avec d'autres professionnels de la santé",
  'respect des protocoles médicaux et des normes de sécurité',
  'rédaction de rapports et tenue de dossiers médicaux',
  'formation continue et développement professionnel',
  'empathie et soutien psychologique'],
 ['diagnostic médical',
  'traitement des maladies et des blessures',
  'prescription de médicaments',
  'suivi et gestion des patients',
  'intervention chirurgicale',
  'communication avec les patients et les familles',
  "collaboration avec d'autres professionnels de la santé",
  'recherche médicale et développement de nouvelles thérapies',
  'formation continue et participation à des conférences',
  'éthique médicale et respect des normes professionnelles'],
 ['assistance aux médecins et infirmières',
  'préparation des patients pour les consultations',
  'prise de rendez-vous et gestion des dossiers médicaux',
  'mesure de signes vitaux et collecte de données médicales',
  'administration de traitements et de médicaments',
  'accueil et orientation des patients',
  'communication avec les patients et leurs familles',
  "collaboration avec l'équipe médicale",
  'respect de la confidentialité et des protocoles',
  'formation continue et développement professionnel'],
 ['délivrance de médicaments sur ordonnance',
  'conseils pharmaceutiques aux patients',
  'gestion des stocks et des commandes',
  'préparation de médicaments',
  'vérification des interactions médicamenteuses',
  'surveillance des effets secondaires et des réactions allergiques',
  'communication avec les médecins et autres professionnels de la santé',
  'promotion de la santé et prévention des maladies',
  'respect des normes de sécurité et de confidentialité',
  'formation continue sur les nouveaux médicaments et traitements'],
 ['examen dentaire et diagnostic',
  'soins dentaires préventifs et curatifs',
  'traitement des caries et des infections dentaires',
  'extraction des dents et chirurgie buccale',
  'prothèses dentaires et implants',
  'orthodontie et correction des malocclusions',
  "conseils d'hygiène bucco-dentaire",
  'gestion des douleurs et des urgences dentaires',
  'communication avec les patients de tous âges',
  'formation continue sur les nouvelles techniques et technologies dentaires'],
 ['évaluation et diagnostic psychologique',
  'thérapie individuelle et de groupe',
  'conseils en santé mentale et bien-être',
  'gestion du stress et des émotions',
  'traitement des troubles psychologiques et des addictions',
  'support et orientation professionnelle',
  'intervention en cas de crise ou de traumatisme',
  'communication empathique et non-jugeante',
  "collaboration avec d'autres professionnels de la santé",
  'confidentialité et respect de la vie privée des patients'],
 ['intervention chirurgicale',
  'diagnostic et planification des opérations',
  "utilisation d'équipements médicaux et chirurgicaux",
  'gestion des risques et des complications',
  'soins post-opératoires et suivi des patients',
  'communication avec les patients et leurs familles',
  "collaboration avec d'autres chirurgiens et spécialistes",
  'formation continue sur les techniques chirurgicales',
  "respect des normes de sécurité et d'hygiène",
  'éthique médicale et consentement éclairé des patients'],
 ["collecte et préparation d'échantillons",
  'analyse et tests de laboratoire',
  "utilisation d'équipements de laboratoire",
  'interprétation de résultats',
  'documentation et tenue de dossiers',
  'respect des protocoles et des normes de sécurité',
  'maintenance et étalonnage des instruments',
  "collaboration avec d'autres techniciens et chercheurs",
  'communication des résultats aux professionnels de la santé',
  'formation continue sur les nouvelles techniques et technologies'],
 ["interprétation d'images médicales",
  "utilisation de technologies d'imagerie (radiographie, IRM, scanner)",
  'diagnostic et identification des anomalies',
  'communication des résultats aux médecins',
  "collaboration avec d'autres radiologues et techniciens",
  'gestion des dossiers et des données patients',
  'respect des normes de sécurité et de confidentialité',
  'formation continue sur les avancées en radiologie',
  'éthique médicale et consentement éclairé'],
 ["administration d'anesthésie générale et locale",
  "surveillance des signes vitaux pendant l'anesthésie",
  'gestion de la douleur post-opératoire',
  'prévention et traitement des complications',
  'communication avec les patients et les chirurgiens',
  "collaboration avec d'autres professionnels de la santé",
  "prise de décisions rapides en situation d'urgence",
  'maintenance et étalonnage des équipements',
  'formation continue sur les nouvelles techniques anesthésiques',
  'respect des protocoles et des normes de sécurité'],
 ["conception et développement de systèmes d'énergie renouvelable",
  "installation et maintenance d'équipements solaires et éoliens",
  "étude d'impact environnemental et évaluation des ressources",
  "optimisation de l'efficacité énergétique",
  "gestion de projets d'énergie renouvelable",
  'utilisation de logiciels de modélisation et de simulation',
  'respect des normes de sécurité et de conformité',
  'analyse économique et financière des projets',
  'communication avec les parties prenantes',
  'veille technologique sur les avancées en énergies renouvelables'],
 ['conception et développement de systèmes photovoltaïques',
  'étude de faisabilité et dimensionnement des installations',
  'utilisation de logiciels de simulation solaire',
  'installation et maintenance de panneaux solaires',
  'suivi de performance et diagnostic des systèmes',
  'gestion de projets solaires',
  'collaboration avec des ingénieurs et des techniciens',
  'veille technologique sur les avancées en énergie solaire',
  'formation continue sur les nouvelles technologies',
  'respect des normes de sécurité et environnementales'],
 ["installation et maintenance d'éoliennes",
  'diagnostic et réparation des équipements',
  "utilisation d'outils de mesure et de diagnostic",
  'gestion de projets éoliens',
  'travail en hauteur et en extérieur',
  'collaboration avec des ingénieurs et des équipes techniques',
  'respect des protocoles de sécurité',
  'formation continue sur les nouvelles technologies éoliennes',
  'documentation des interventions et des données',
  'communication avec les clients et les partenaires'],
 ['exploration et production de pétrole et de gaz',
  'analyse géologique et évaluation des réserves',
  "conception de puits de forage et d'équipements",
  'utilisation de techniques de forage et de fracturation',
  "gestion de projets d'extraction",
  'optimisation de la production et du rendement',
  "gestion des risques et de l'impact environnemental",
  'respect des normes de sécurité et de réglementation',
  'communication avec les équipes sur le terrain',
  'formation continue sur les nouvelles technologies et méthodes'],
 ['maintenance préventive et corrective des équipements',
  'diagnostic et dépannage des pannes',
  "utilisation d'outils et d'équipements de mesure",
  'gestion des stocks de pièces de rechange',
  'planification des interventions de maintenance',
  'documentation des activités de maintenance',
  'collaboration avec des équipes techniques',
  'respect des normes de sécurité et des procédures',
  'formation continue sur les nouvelles technologies',
  'communication avec les clients et les utilisateurs'],
 ['sécurité et sûreté nucléaire',
  'contrôle des réacteurs et des installations',
  'gestion des déchets radioactifs',
  'maintenance et réparation des équipements',
  'formation du personnel et supervision des opérations',
  "analyse des risques et planification d'urgence",
  'conformité aux réglementations et normes',
  'recherche et développement en énergie nucléaire',
  'communication avec les parties prenantes',
  'veille technologique sur les avancées nucléaires'],
 ['évaluation des besoins énergétiques',
  'audit et analyse de la consommation énergétique',
  "proposition de solutions d'efficacité énergétique",
  'mise en œuvre de mesures de réduction de la consommation',
  "gestion de projets d'optimisation énergétique",
  'utilisation de logiciels de simulation énergétique',
  'communication avec les clients et les partenaires',
  "formation et sensibilisation à l'économie d'énergie",
  'évaluation de la performance énergétique',
  'respect des réglementations et des normes environnementales'],
 ['collecte et analyse de données énergétiques',
  'modélisation et prévision de la consommation',
  'évaluation de la performance des installations',
  "identification des opportunités d'économie d'énergie",
  "rapports et recommandations pour l'efficacité énergétique",
  "utilisation de logiciels d'analyse et de simulation",
  'communication avec les clients et les décideurs',
  'suivi des tendances et des innovations énergétiques',
  'formation continue sur les méthodes et les technologies',
  'collaboration avec des équipes techniques et commerciales'],
 ['planification et organisation de projets énergétiques',
  'gestion des ressources et des budgets',
  'suivi des échéanciers et des objectifs',
  'coordination des équipes et des intervenants',
  'communication avec les clients et les parties prenantes',
  'gestion des risques et des imprévus',
  "rapports d'avancement et documentation des projets",
  'respect des réglementations et des normes',
  'évaluation de la performance et des résultats',
  'formation continue en gestion de projet et en énergie'],
 ['installation et maintenance de systèmes de chauffage',
  'ventilation et climatisation',
  'diagnostic et dépannage des systèmes HVAC',
  "utilisation d'outils de mesure et de test",
  'lecture de plans et de schémas techniques',
  'gestion des pièces de rechange et des stocks',
  'respect des normes de sécurité et de qualité',
  'formation des utilisateurs finaux',
  'collaboration avec des équipes techniques',
  'communication avec les clients et les fournisseurs'],
 ['tenue de livres et comptabilité générale',
  'analyse des états financiers',
  'déclarations fiscales et rapports réglementaires',
  'gestion des budgets et des prévisions',
  'audit comptable et contrôle interne',
  'gestion de trésorerie et recouvrement',
  'utilisation de logiciels de comptabilité',
  'communication avec les clients et les auditeurs',
  'formation continue sur les normes comptables',
  "respect de la confidentialité et de l'éthique professionnelle"],
 ['analyse des marchés financiers',
  'évaluation des investissements et des risques',
  'prévisions financières et modélisation',
  "rédaction de rapports d'analyse financière",
  "gestion de portefeuille et allocation d'actifs",
  "utilisation de logiciels d'analyse financière",
  'communication avec les investisseurs et les clients',
  'formation continue sur les marchés et les instruments financiers',
  'collaboration avec des équipes commerciales et de gestion',
  'respect des réglementations et des normes professionnelles'],
 ['structuration et négociation de transactions financières',
  'évaluation des entreprises et des projets',
  'montage financier et due diligence',
  'analyse des risques et des opportunités',
  'gestion de portefeuille et de risque',
  'communication avec les clients et les partenaires',
  "utilisation de modèles financiers et d'outils d'analyse",
  'formation continue sur les marchés financiers et les produits dérivés',
  'collaboration avec des équipes juridiques et commerciales',
  'respect des réglementations et des normes éthiques'],
 ['analyse des performances et des écarts',
  'prévisions budgétaires et planification stratégique',
  'reporting financier et tableau de bord',
  'optimisation des coûts et des investissements',
  'gestion des budgets et des prévisions',
  'évaluation des processus et des projets',
  'communication avec les départements opérationnels',
  'utilisation de logiciels de gestion et de business intelligence',
  'formation continue sur les techniques de contrôle de gestion',
  'collaboration avec des équipes financières et opérationnelles'],
 ['audit financier et opérationnel',
  'évaluation des risques et des contrôles internes',
  'vérification de la conformité aux normes et réglementations',
  'analyse des processus et des données',
  'révision des états financiers et des rapports',
  'communication des résultats et recommandations',
  "utilisation de logiciels d'audit et de gestion des risques",
  "formation continue sur les normes et les pratiques d'audit",
  "collaboration avec des équipes d'audit interne et externe",
  "respect de l'éthique et de la confidentialité professionnelle"],
 ["gestion de portefeuille et allocation d'actifs",
  "analyse des marchés financiers et des opportunités d'investissement",
  'sélection et suivi des placements',
  'relation clientèle et gestion de patrimoine',
  "communication des stratégies d'investissement",
  'utilisation de logiciels de gestion de fonds',
  'formation continue sur les tendances et les innovations en finance',
  "collaboration avec des équipes de recherche et d'analyse",
  'respect des réglementations et des normes éthiques',
  'performance et résultats financiers'],
 ['analyse de la situation financière des clients',
  'planification patrimoniale et fiscale',
  "recommandations d'investissement et de gestion de patrimoine",
  'élaboration de stratégies financières personnalisées',
  'suivi et ajustement des plans financiers',
  'communication des produits financiers et des risques',
  'relation clientèle et fidélisation',
  "utilisation d'outils de simulation financière",
  'formation continue sur les produits et les marchés financiers',
  'collaboration avec des équipes de conseillers et de vente'],
 ['recherche et analyse de produits financiers',
  "négociation et exécution d'ordres d'achat et de vente",
  "conseils sur les stratégies d'investissement",
  'gestion de portefeuille et de risque',
  'relation clientèle et prospection commerciale',
  'suivi des marchés financiers et des tendances',
  "utilisation de plateformes de trading et d'analyse",
  'formation continue sur les instruments financiers',
  'respect des réglementations et des normes professionnelles',
  'performance et résultats commerciaux'],
 ['évaluation de la solvabilité des emprunteurs',
  'analyse des états financiers et des données économiques',
  "notation de crédit et décision d'octroi",
  'gestion des risques et des garanties',
  'communication avec les clients et les partenaires bancaires',
  "utilisation de modèles de scoring et d'analyse",
  "formation continue sur les méthodes d'analyse de crédit",
  'collaboration avec des équipes commerciales et juridiques',
  'respect des politiques de crédit et des normes réglementaires',
  'performance et qualité des décisions de crédit'],
 ['identification et évaluation des risques financiers',
  'élaboration de politiques et de stratégies de gestion des risques',
  'mise en place de dispositifs de contrôle et de suivi',
  'analyse des scénarios et des impacts',
  'communication des recommandations aux décideurs',
  "utilisation d'outils d'analyse et de modélisation des risques",
  'formation continue sur les pratiques de gestion des risques',
  'collaboration avec des équipes de gestion et de conformité',
  'respect des réglementations et des normes internes',
  'performance et efficacité des dispositifs de gestion des risques'],
 ['apprentissage automatique (Machine Learning)',
  'réseaux neuronaux et deep learning',
  'traitement du langage naturel (NLP)',
  'vision par ordinateur',
  'reconnaissance de formes et de motifs',
  'algorithmes génétiques et optimisation',
  'systèmes experts et logique floue',
  'apprentissage non supervisé et clustering',
  "utilisation de bibliothèques d'IA (TensorFlow, PyTorch)",
  "développement d'applications et de solutions IA"],
 ['technologie de registre distribué (DLT)',
  'cryptomonnaies et tokens',
  'contrats intelligents (smart contracts)',
  'minage et validation de transactions',
  'consensus et preuve de travail/autorité/participation',
  'interopérabilité entre blockchains',
  'applications décentralisées (dApps)',
  'sécurité et confidentialité des données',
  'utilisation de frameworks blockchain (Ethereum, Hyperledger)',
  'développement de solutions blockchain'],
 ['conception et modélisation de robots',
  'programmation de robots et de systèmes embarqués',
  'capteurs et actionneurs',
  'navigation autonome et SLAM (Simultaneous Localization And Mapping)',
  'interaction homme-machine',
  'intelligence artificielle embarquée',
  "vision par ordinateur et reconnaissance d'objets",
  'systèmes de contrôle et de planification de mouvement',
  'applications industrielles et domestiques de la robotique',
  "collaboration avec des équipes d'ingénieurs et de chercheurs"],
 ['biologie moléculaire et cellulaire',
  'génétique et génomique',
  'culture de cellules et fermentation',
  "ingénierie génétique et manipulation de l'ADN",
  'bioprocédés et biofabrication',
  'analyse bioinformatique et modélisation',
  'diagnostic médical et biotechnologies médicales',
  'développement de médicaments et thérapies géniques',
  'biomatériaux et bio-impression 3D',
  'respect des normes de sécurité et des bonnes pratiques'],
 ['développement logiciel',
  'programmation',
  "conception d'architecture",
  'gestion de projet',
  'test et débogage',
  'maîtrise des langages de programmation (Java, Python, C++)',
  'connaissance des frameworks (Spring, Django, .NET)',
  'utilisation des outils de versioning (Git, SVN)',
  'résolution de problèmes complexes',
  'collaboration en équipe'],
 ['développement web',
  "conception d'interfaces utilisateur",
  'langages de programmation (HTML, CSS, JavaScript)',
  'frameworks web (React, Angular, Vue.js)',
  'bases de données (SQL, MongoDB)',
  'maîtrise des outils de développement (Visual Studio Code, Webpack)',
  'gestion de projets Agile',
  "sensibilité au design et à l'expérience utilisateur",
  'résolution de problèmes techniques',
  'veille technologique'],
 ['analyse de données',
  'exploration et nettoyage de données',
  'modélisation statistique et machine learning',
  'programmation en langage R et Python',
  'utilisation de bibliothèques (NumPy, pandas, scikit-learn)',
  'visualisation de données (Matplotlib, Seaborn, Plotly)',
  "compréhension des algorithmes d'apprentissage automatique",
  'traitement de données massives (Big Data)',
  'résolution de problèmes complexes',
  'communication des résultats'],
 ['sécurité informatique',
  'détection et prévention des intrusions',
  'analyse de vulnérabilités',
  'gestion des risques',
  'cryptographie',
  'sécurité des réseaux',
  "tests d'intrusion",
  'conformité réglementaire (RGPD, ISO 27001)',
  'sensibilisation à la sécurité',
  'gestion des incidents de sécurité'],
 ['administration système',
  'configuration et maintenance des réseaux',
  "sécurité des systèmes d'information",
  'supervision et surveillance des infrastructures',
  "résolution d'incidents réseau",
  'virtualisation (VMware, Hyper-V)',
  "gestion des sauvegardes et de la reprise d'activité",
  "gestion des utilisateurs et des droits d'accès",
  'veille technologique',
  'collaboration avec les équipes techniques'],
 ["développement d'applications mobiles",
  'langages de programmation (Java, Kotlin, Swift)',
  "conception d'interfaces utilisateur mobiles",
  'utilisation des SDK (Android, iOS)',
  "déploiement sur les stores d'applications",
  'test et débogage sur différentes plateformes',
  'optimisation des performances',
  'sécurité des applications mobiles',
  'collaboration avec les équipes de design et de backend',
  'adaptabilité aux évolutions des OS mobiles'],
 ['architecture informatique',
  'conception de solutions techniques',
  'intégration de systèmes',
  'scalabilité et haute disponibilité',
  'sécurité des systèmes',
  'virtualisation et cloud computing',
  'gestion des bases de données',
  'optimisation des performances',
  'veille technologique',
  'communication et présentation de solutions'],
 ["design d'interfaces utilisateur (UI)",
  'expérience utilisateur (UX)',
  'wireframing et prototypage',
  'conception graphique',
  'utilisabilité et accessibilité',
  'ergonomie des interfaces',
  'tests utilisateurs',
  'outils de design (Sketch, Adobe XD, Figma)',
  'collaboration avec les développeurs et les chefs de projet',
  'créativité et sensibilité artistique'],
 ['analyse des besoins',
  'étude de faisabilité',
  'spécifications fonctionnelles et techniques',
  'modélisation des processus métier',
  'gestion de projet',
  'tests et recettes',
  'formation des utilisateurs',
  'support technique',
  "résolution d'incidents",
  'documentation des systèmes'],
 ['cloud computing',
  'architecture cloud',
  'migration vers le cloud',
  'services cloud (AWS, Azure, Google Cloud)',
  'automatisation et orchestration',
  'sécurité cloud',
  'gestion des coûts',
  'performances et scalabilité',
  'administration des plateformes cloud',
  'collaboration en équipe']]

In [10]:
import json

with open('competences.json', 'w', encoding='utf-8') as f:
    json.dump(liste_competences_rome, f, ensure_ascii=False, indent=4)

# save the json file


In [11]:
df = pd.read_csv('../data/concatenated_offers.csv')

C:\Users\Moham\AppData\Local\Temp\ipykernel_15748\2741407724.py:1: DtypeWarning: Columns (39,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/concatenated_offers.csv')


In [12]:
df_processed = filtrer_competences_df(df, liste_competences_rome)

In [13]:
# save the processed dataframe
df_processed.to_csv('../data/concatenated_offers_processed.csv', index=False)

# Word2Vec Model Application

In [14]:
word2vec_model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


C:\Users\Moham\AppData\Local\Temp\ipykernel_15748\580439706.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  word2vec_model = FT_gensim.load_fasttext_format('C:\\Users\\Moham\\Documents\\data_pole_emploi\\Data_Pole_Emploi\\cc.fr.300.bin.gz')


In [40]:
# Define your Word2Vec calculation function
def calculate_average_word2vec_optimized(token_lists, model):
    num_features = model.vector_size
    feature_vecs = np.zeros((len(token_lists), num_features), dtype="float32")
    for idx, tokens in enumerate(token_lists):
        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        if valid_tokens:
            model_vocabs = np.array([model.wv[token] for token in valid_tokens])
            feature_vecs[idx] = np.mean(model_vocabs, axis=0)
    return feature_vecs

# Load your Word2Vec model, adjust the path as needed
df_processed = pd.read_csv('../data/concatenated_offers_processed.csv')

# Apply the mean vector calculation to the processed DataFrame
df_processed['mean_vector'] = df_processed['competence_processed'].apply(
    lambda tokens: calculate_average_word2vec_optimized([tokens], word2vec_model)[0]
)
# Use your PDF reading and tokenization functions to prepare the resume's text
#texte = lirePDF("C:\\Users\\Moham\\Desktop\\cv_fr.pdf")

def  liretexte(texte):
    texte= re.sub(r'http\S+', ' ', texte) # enlever liens(texte)

    texte= re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', texte) # enlever adresses email

    texte= re.sub(r'[0-9()+-]', ' ', texte) #enlever numeros et symboles

    texte= re.sub(r'[^\w\s]', ' ', texte) # enlever symboles
    
    return texte
texte= """ILYESS GUERROUJ
Ingénieur avant-vente ∼ Ingénieur solution
Ó 06 66 55 97 28
½ Paris, Genève
R ilyessguerrouj@gmail.com
° /ilyess guerrouj
RÉSUMÉ
Ingénieur solution chez Salesforce, avec un fort intérêt pour .
L'analyse de données, l'IA et le business.
COMPÉTENCES
Compétences informatiques : Python, SQL, Java, C++, C.
Technologies : Salesforce, Tableau, Commerce
cloud, Microsoft Excel, AWS.
EXPÉRIENCE
09/22 - 09/24 Ingénieur avant-vente/solution Salesforce
- Maîtrise de la plateforme Salesforce permettant une personnalisation adaptée aux attentes des clients B2B.
- Conseiller les clients du commerce de détail dans la phase d'avant-vente implique de montrer la valeur de Salesforce et de démontrer l'utilisation de la plateforme concernant les données, l'IA et l'e-commerce.
l'utilisation de la plateforme concernant les données, l'IA et le commerce électronique à travers les solutions CRM.
- Collecte de données, création d'un jeu collaboratif de références clients permettant leur réutilisation comme source d'inspiration pour de futurs clients potentiels.
pour les futurs clients potentiels.
23/02 - 23/06 Analyste commercial Evolukid
- Réalisation d'un prototype (logiciel) qui, grâce à l'intelligence artificielle, permettra aux cantines de Marseille de ne plus gaspiller de nourriture et d'offrir les meilleurs services possibles.
de ne plus gaspiller la nourriture et d'offrir les meilleurs repas possibles. Analyse des bases de données collectées par la ville, nettoyage des données et création de tableaux de bord afin d'améliorer la qualité des repas.
nettoyer les données et créer des tableaux de bord pour prendre les meilleures décisions possibles. .
03/19 - aujourd'hui PDG et cofondateur Digiweb
- Agence de développement web : Graphisme, DigitalMarketing, Ecommerce. J'ai travaillé avec une entreprise de prêt-à-porter féminin à Londres et j'ai été chauffeur d'un camion de transport.
prêt-à-porter féminin à Londres et une entreprise d'auto-école à Paris.
10/19 - 01/23 Responsable de ma boutique en ligne Manga Industry(eProShopping)
- Conceptualisation de ma boutique en ligne : vente de produits autour du manga japonais.
*** Traduit avec www.DeepL.com/Translator (version gratuite) ***

- Conceptualisé ma boutique en ligne : Vente de produits autour de l'animation japonaise. Gestion des clients et des fournisseurs
gestion des clients et fournisseurs en France et à l'étranger. Promotion sur les réseaux sociaux : 700 visites sur le site par mois.
FORMATION
09/21 - 09/24 Msc in Engineering, Major in Data science Ecole Centrale de Marseille
Parcours : Conception, mise en œuvre et maîtrise d'ouvrage de systèmes d'information. Science des données.
Gestion de projet en transformation digitale, Business Services.Probabilités/statistiques, théorie des graphes.
02/23 - 06/23 Mémoire de Master en Science des données Imperial College London
Amélioration d'un outil d'analyse de données. Il s'agissait d'utiliser des méthodes d'IA pour analyser et prédire de grandes bases de données de séries temporelles.
de grandes bases de données de séries temporelles. Collaboration étroite avec une université en Chine.
09/18 - 09/21 Licence de physique et mathématiques-informatique Université Paris Saclay
Physique quantique, physique statistique. Mathématiques (analyse, algèbre linéaire). Algorithmique.
CERTIFICATIONS
2024 (en cours) AWS Certified Cloud Practitioner AmazonWeb Services
Solides compétences techniques en matière d'infrastructure et de services cloud AWS.
2023 Salesforce Business Analyst Salesforce
Spécialisé dans l'analyse et l'optimisation des processus commerciaux CRM afin d'améliorer l'efficacité et la productivité.
2023 Administrateur certifié Salesforce Salesforce
Gestion des données, CRM, compétences en matière de plateforme Salesforce.
2022 Microsoft Certified : Azure Fundamentals, AZ-900 Microsoft
Concept de nuage, analyse de données, service de nuage et sécurité.
PROJETS ET BÉNÉVOLAT
10/20 - aujourd'hui Projet d'égalité des chances S

10/20 - aujourd'hui Projet d'égalité des chances Association de soutien scolaire Orly
Formation d'un groupe de 6 élèves, chaque semaine, suivi et accompagnement vers la réussite scolaire.
12/20 - aujourd'hui Projet humanitaire LaPointeCharity, Choisy-le-Roi
Prendre soin des plus démunis en France. Accès à l'eau potable au Togo et au Bangladesh.
LANGUES
Français - Natif/ Anglais - Courant/ Espagnol - Professionnel
 """
texte = liretexte(texte)
path_cv = "C:\\Users\\Moham\\Downloads\\CV-camille.pdf"
#texte = lirePDF(path_cv)

competences_texte = filtrer_competences_texte(texte, liste_competences_rome) 

# Calculate the mean vector for the resume
mean_vector_texte = calculate_average_word2vec_optimized([competences_texte], word2vec_model)[0]

# Define the debug and cosine similarity calculation functions
def debug_vector_sizes(vec1, vec2):
    if len(vec1) != len(vec2):
        # Adjust size if necessary
        if len(vec1) < len(vec2):
            vec1 = np.append(vec1, np.zeros(len(vec2) - len(vec1)))
        else:
            vec2 = np.append(vec2, np.zeros(len(vec1) - len(vec2)))
    return vec1, vec2

def calculate_cosine_similarity(vec1, vec2):
    vec1, vec2 = debug_vector_sizes(vec1, vec2)
    if np.linalg.norm(vec1) != 0 and np.linalg.norm(vec2) != 0:
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return 0

# Apply cosine similarity calculation to the DataFrame
df_processed['cosine_similarity'] = df_processed['mean_vector'].apply(lambda x: calculate_cosine_similarity(x, mean_vector_texte))

# Sort and display the most similar job offers
df_processed = df_processed.sort_values(by='cosine_similarity', ascending=False)
df_processed.head(10)



#df_processed['word2vec_vector'] = calculate_average_word2vec_optimized(df_processed['competence_processed'].tolist(), word2vec_model)

C:\Users\Moham\AppData\Local\Temp\ipykernel_15748\516339719.py:13: DtypeWarning: Columns (21,39,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_processed = pd.read_csv('../data/concatenated_offers_processed.csv')


,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,contact.telephone,origineOffre.partenaires,conditionExercice,agence.telephone,Secteur,Date,Unnamed: 0,competence_processed,mean_vector,cosine_similarity
49532,169LFPC,"Préparer petits plats, planchettes charcuterie...","Vous aurez en charge de préparer des salades, ...",2024-02-14T14:21:01.000Z,2024-02-14T21:29:18.000Z,G1602,Personnel de cuisine,Commis / Commise de cuisine,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,10.0,2024-01-16 to 2024-02-15,92.0,"['différent', 'produit', 'prépar']","[-0.14990555, -0.07026318, -0.0866438, -0.0707...",0.457040
4403,160LLBX,Maçon / Maçonne,- Descriptif :\n- mettre en oeuvre des structu...,2023-08-28T09:13:39.000Z,2024-01-02T11:38:41.000Z,F1703,Maçonnerie,Maçon / Maçonne,MIS,Mission intérimaire - 5 Jour(s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['extérieur', 'appliqu', 'différent', 'autr', ...","[-0.13316259, -0.074861415, -0.08202138, -0.07...",0.456993
3881,1433615,Bardeur(euse) (Rouen) H/F,Intégré(e) aux équipes de production de l'agen...,2023-08-31T16:28:19.000Z,2024-01-05T10:04:15.000Z,F1502,Montage de structures métalliques,Bardeur / Bardeuse,CDI,Contrat à durée indéterminée,...,NaN,"[{'nom': 'DIRECTEMPLOI', 'url': 'https://www.d...",NaN,NaN,NaN,NaN,NaN,"['integr', 'product', 'équip', 'différent', 'p...","[-0.13485886, -0.077542074, -0.072681464, -0.0...",0.455600
7430,8868867,Maçon / Maçonne (H/F),Description du poste :\nVos missions :\n- mett...,2023-07-15T15:50:22.000Z,2024-01-04T11:05:35.000Z,F1703,Maçonnerie,Maçon / Maçonne,MIS,Mission intérimaire - 12 Mois,...,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo...",NaN,NaN,NaN,NaN,NaN,"['œuvr', 'différent', 'extérieur', 'autr', 'pr...","[-0.12987965, -0.07307839, -0.08011029, -0.070...",0.455497
42412,169DSNR,Maçon / Maçonne,Les missions:\n\n-\tMettre en œuvre des struct...,2024-02-09T11:29:59.000Z,2024-02-15T09:06:36.000Z,F1703,Maçonnerie,Maçon / Maçonne,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,41.0,2024-01-16 to 2024-02-15,4732.0,"['extérieur', 'appliqu', 'œuvr', 'différent', ...","[-0.12987965, -0.073078394, -0.08011029, -0.07...",0.455497
31069,170RLXY,Couvreur / Couvreuse,Vous effectuez la préparation et la pose des é...,2024-03-07T10:48:47.000Z,2024-03-07T10:48:47.000Z,F1610,Pose et restauration de couvertures,Couvreur / Couvreuse,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,41.0,2024-02-10 to 2024-03-11,NaN,"['sécurité', 'préparat', 'sécur', 'prépar']","[-0.15190561, -0.07601257, -0.11748505, -0.109...",0.455355
15947,166KXHS,Ouvrier agricole polyvalent productions végéta...,Ouvrier agricole polyvalent pour 3 ateliers de...,2023-12-16T06:39:13.000Z,2024-01-10T13:43:51.000Z,A1414,Horticulture et maraîchage,Ouvrier / Ouvrière agricole en maraîchage-hort...,SAI,Contrat travail saisonnier - 6 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['travail', 'extérieur', 'cultur', 'product', ...","[-0.12844257, -0.07135978, -0.06945987, -0.062...",0.454927
46428,169HYMQ,PREPARATEUR DE VEHICULES (H/F),nous recherchons un préparateur de véhicules s...,2024-02-13T10:08:43.000Z,2024-02-15T15:05:36.000Z,I1604,Mécanique automobile et entretien de véhicules,Préparateur / Préparatrice de véhicules automo...,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,77.0,2024-01-16 to 2024-02-15,8926.0,"['extérieur', 'vérific', 'sécurité', 'sécur', ...","[-0.14263257, -0.06022513, -0.05802731, -0.054...",0.454806
50456,169LFCV,Chauffeur routier / Chauffeuse routière,"Afin de renforcer nos équipes, nous recrutons ...",2024-02-14T14:09:17.000Z,2024-02-15T09:58:00.000Z,N4101,Conduite de transport de marchandises sur long...,Chauffeur routier / Chauffeuse routière,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,49.0,2024-01-16 to 2024-02-15,4933.0,"['départ', 'équip', 'différent', ""d'autr""]","[-0.121124186, -0.060004577, -0.07124845, -0.0...",0.454531
31081,170QTDV,Ravaleur / Ravaleuse,Nous cherchons un(e

# TF-IDF Application and Cosine Similarity

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your 'liste_competences_rome' is a flat list of all unique competencies.
all_competences = [item for sublist in liste_competences_rome for item in sublist]
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(all_competences)

def apply_tfidf_vectorizer(text_list, vectorizer):
    # Transform list of texts into TF-IDF vectors
    tfidf_matrix = vectorizer.transform([' '.join(text_list)])
    return tfidf_matrix.mean(axis=0)  # You might want to average here if your logic requires, but typically you just return the matrix

# Update DataFrame with TF-IDF vectors
df_processed['tfidf_vector'] = df_processed['competence_processed'].apply(lambda x: apply_tfidf_vectorizer(x, tfidf_vectorizer))


In [42]:
# Assuming 'competences_texte' is a list of tokens (words)
# Convert each list of tokens (single document) into a string
resume_text_list = [' '.join(competences_texte)]  # Now resume_text_list is a list of one string

# Now apply your TF-IDF vectorization using this updated list
resume_tfidf_vector = apply_tfidf_vectorizer(resume_text_list, tfidf_vectorizer)


In [43]:
# Ensure vec1 and vec2 are compatible for cosine similarity calculations
# Assuming vec1 and vec2 are scipy sparse matrices or numpy arrays
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse

def calculate_cosine_similarity_tfidf(vec1, vec2):
    # Convert vec1 and vec2 to dense arrays if they are in sparse format
    if scipy.sparse.issparse(vec1):
        vec1 = vec1.toarray()
    if scipy.sparse.issparse(vec2):
        vec2 = vec2.toarray()
    
    # Convert vec1 and vec2 to NumPy arrays if they are np.matrix instances
    vec1 = np.asarray(vec1)
    vec2 = np.asarray(vec2)

    # Now you can safely compute cosine similarity
    return cosine_similarity(vec1, vec2)


# Apply cosine similarity calculation between job descriptions and the resume
df_processed['cosine_similarity_tfidf'] = df_processed['tfidf_vector'].apply(lambda x: calculate_cosine_similarity_tfidf(x, resume_tfidf_vector))


In [44]:
# Sort and display the most similar job offers based on TF-IDF cosine similarity
df2_processed = df_processed.sort_values(by='cosine_similarity_tfidf', ascending=False)

df2_processed.head(10)

,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,conditionExercice,agence.telephone,Secteur,Date,Unnamed: 0,competence_processed,mean_vector,cosine_similarity,tfidf_vector,cosine_similarity_tfidf
49532,169LFPC,"Préparer petits plats, planchettes charcuterie...","Vous aurez en charge de préparer des salades, ...",2024-02-14T14:21:01.000Z,2024-02-14T21:29:18.000Z,G1602,Personnel de cuisine,Commis / Commise de cuisine,CDI,Contrat à durée indéterminée,...,NaN,NaN,10.0,2024-01-16 to 2024-02-15,92.0,"['différent', 'produit', 'prépar']","[-0.14990555, -0.07026318, -0.0866438, -0.0707...",0.457040,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
25890,3823318,Conducteur (H/F),Description du poste :\nRattaché(e) au respons...,2023-10-08T08:46:12.000Z,2024-01-04T11:58:48.000Z,N4103,Conduite de transport en commun sur route,Conducteur / Conductrice machiniste,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,"['form', 'qualité', 'conformité', 'perform', '...","[-0.11727249, -0.05941013, -0.030815857, -0.07...",0.426781,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
2570,2142712,INGENIEUR RESPONSABLE TECHNIQUE SECURITE (H/F),[15374] CENTRE HOSPITALIER DE GONESSE - ESPACE...,2023-09-12T20:23:22.000Z,2023-11-13T18:25:52.000Z,K2402,"Recherche en sciences de l'univers, de la mati...",Ingénieur / Ingénieure de recherche biomédicale,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,"['rédact', 'cod', 'œuvr', ""d'acces"", 'sécurité...","[-0.10869061, -0.05673051, -0.031126602, -0.07...",0.426745,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
31498,170VPPN,vendeur/se VN h/f TAF,Votre mission consistera à :\n Accueillir et c...,2024-03-11T09:26:13.000Z,2024-03-11T09:30:33.000Z,N1103,Magasinage et préparation de commandes,Magasinier vendeur / Magasinière vendeuse,CDI,Contrat à durée indéterminée,...,NaN,NaN,45.0,2024-02-10 to 2024-03-11,NaN,"['réparat', 'piec', 'répar', 'pièc', 'qualité'...","[-0.11513941, -0.064363845, -0.03941149, -0.07...",0.426745,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
31496,170VPSS,vendeur/se VO h/f TAF,Votre mission consistera à :\n Accueillir et c...,2024-03-11T09:30:00.000Z,2024-03-11T09:31:02.000Z,N1103,Magasinage et préparation de commandes,Magasinier vendeur / Magasinière vendeuse,CDI,Contrat à durée indéterminée,...,NaN,NaN,45.0,2024-02-10 to 2024-03-11,NaN,"['réparat', 'piec', 'répar', 'pièc', 'qualité'...","[-0.11513941, -0.064363845, -0.03941149, -0.07...",0.426745,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
42988,169MLTY,Vendeur / Vendeuse comptoir de matériels et éq...,Description du poste\nVéritable spécialiste da...,2024-02-15T11:52:21.000Z,2024-02-15T16:16:38.000Z,D1213,Vente en gros de matériel et équipement,Vendeur / Vendeuse comptoir de matériels et éq...,CDI,Contrat à durée indéterminée,...,NaN,NaN,46.0,2024-01-16 to 2024-02-15,5308.0,"['qualité', 'conformité', 'informat', 'opérat'...","[-0.11788009, -0.061933793, -0.029816752, -0.0...",0.426744,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
40146,169FMVM,Gestionnaire de flux logitique (H/F),"Rattaché(e) au Responsable Supply Chain, vous ...",2024-02-09T16:25:22.000Z,2024-02-12T08:27:29.000Z,N1301,Conception et organisation de la chaîne logist...,Chargé / Chargée de gestion logistique,CDI,Contrat à durée indéterminée,...,NaN,NaN,23.0,2024-01-16 to 2024-02-15,2346.0,"['sécurité', 'opérat', ""d'optimis"", 'relat', '...","[-0.11284794, -0.0666498, -0.031825434, -0.076...",0.426744,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,[[0.0]]
35923,170VMHF,Aide ménager / aide ménagère (H/F),Le service à la personne est un domaine qui vo...,2024-03-11T06:07:16.000Z,2024-03-11T06:07:17.000Z,K1304,Services domestiques,Aide ménager / ménagère à domicile,CDI,Contrat à durée indéterminée,...,NaN,NaN,81.0,2024-02-10 to 2024-03-11,NaN,"['nouveau', 'professionnell', 'contrat', 'adap...","[-0.12076955, -0.06

In [39]:
# df2 processed and df processed to save as omar_cv_recommendation for word2vec and tfidf only the colums cosine_similarity and cosine_similarity_tfidf title and description and competences

df2_processed = df2_processed[['intitule', 'description','romeLibelle', 'competences', 'cosine_similarity_tfidf']].head(20)
df_processed = df_processed[['intitule', 'description', 'romeLibelle','competences', 'cosine_similarity']].head(20)
df2_processed.to_csv('ilyess_cv_recommendation_tfidf.csv', index=False)

df_processed.to_csv('ilyess_cv_recommendation_tfidf.csv', index=False)

